In [1]:
from IPython.display import display
from matplotlib.ticker import StrMethodFormatter
from matplotlib.ticker import FormatStrFormatter

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import spacy

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.set_option('display.max_colwidth', None)

%matplotlib inline


In [2]:
#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
#for word embedding
import gensim
from gensim.models import Word2Vec

wl = WordNetLemmatizer()
snow = SnowballStemmer('english')


[nltk_data] Downloading package punkt to /home/mikes/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/mikes/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/mikes/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/mikes/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# vd = pd.read_csv("/home/mikes/Windows/Documents/tmp/VAERS data deaths only Edited for NLP for HAD COVID.csv", 
#                          delimiter = ",", 
#                          encoding = "ISO-8859-1")

vd = pd.read_csv("./VAERS data deaths only Edited for NLP for HAD COVID.csv", 
                         delimiter = ",", 
                         encoding = "ISO-8859-1")

In [4]:
vd. columns

Index(['VAERS_ID', 'RECVDATE', 'STATE', 'AGE_YRS', 'CAGE_YR', 'CAGE_MO', 'SEX',
       'RPT_DATE', 'SYMPTOM_TEXT', 'SYMPTOM_TEXT_TARGET', 'DIED', 'DATEDIED',
       'L_THREAT', 'ER_VISIT', 'HOSPITAL', 'HOSPDAYS', 'X_STAY', 'DISABLE',
       'RECOVD', 'VAX_DATE', 'ONSET_DATE', 'NUMDAYS', 'LAB_DATA',
       'LAB_DATA_TARGET', 'V_ADMINBY', 'V_FUNDBY', 'OTHER_MEDS',
       'OTHER_MEDS_TARGET', 'CUR_ILL', 'CUR_ILL_TARGET', 'HISTORY',
       'PRIOR_VAX', 'SPLTTYPE', 'FORM_VERS', 'TODAYS_DATE', 'BIRTH_DEFECT',
       'OFC_VISIT', 'ER_ED_VISIT', 'ALLERGIES'],
      dtype='object')

In [5]:
#convert to lowercase and remove punctuations and characters and then strip
def preprocess(text):
    text = text.lower() #lowercase text
    text=text.strip()  #get rid of leading/trailing whitespace 
    text=re.compile('<.*?>').sub('', text) #Remove HTML tags/markups
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  #Replace punctuation with space. Careful since punctuation can sometime be useful
    text = re.sub('\s+', ' ', text)  #Remove extra space and tabs
    text = re.sub(r'\[[0-9]*\]',' ',text) #[0-9] matches any digit (0 to 10000...)
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) #matches any digit from 0 to 100000..., \D matches non-digits
    text = re.sub(r'\s+',' ',text) #\s matches any whitespace, \s+ matches multiple whitespace, \S matches non-whitespace 
    
    return text

#1. STOPWORD REMOVAL
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)

#2. STEMMING 
# Initialize the stemmer
#snow = SnowballStemmer('english')
def stemming(string):
    a=[snow.stem(i) for i in word_tokenize(string) ]
    return " ".join(a)

#3. LEMMATIZATION
# Initialize the lemmatizer
#wl = WordNetLemmatizer()
 
# This is a helper function to map NTLK position tags
# Full list is available here: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

# Tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

def finalpreprocess(string):
    string = str(string)
    return lemmatizer(stopword(preprocess(string)))
#    return lemmatizer(preprocess(string))

In [6]:
# if "apple" in stopwords.words('english'): print(1)

In [7]:
#for converting sentence to vectors/numbers from word vectors result by Word2Vec
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(next(iter(word2vec.values())))

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

The precision is the ratio tp / (tp + fp) where tp is the number of true positives and fp the number of false positives. The precision is intuitively the ability of the classifier not to label as positive a sample that is negative.

The recall is the ratio tp / (tp + fn) where tp is the number of true positives and fn the number of false negatives. The recall is intuitively the ability of the classifier to find all the positive samples.

The F-beta score can be interpreted as a weighted harmonic mean of the precision and recall, where an F-beta score reaches its best value at 1 and worst score at 0.The F-beta score weights recall more than precision by a factor of beta. beta == 1.0 means recall and precision are equally important.

The support is the number of occurrences of each class in y_true.

If pos_label is None and in binary classification, this function returns the average precision, recall and F-measure if average is one of 'micro', 'macro', 'weighted' or 'samples'.

Read more in the [User Guide](https://scikit-learn.org/stable/modules/model_evaluation.html#precision-recall-f-measure-metrics).

In [8]:

def logistic_regression_tfidf(df_train, df_test, ID, TEXT, TARGET, perc_cutoff = 80):
    if perc_cutoff > 1: perc_cutoff = perc_cutoff/100
    df_train = df_train.loc[df_train[TARGET].notnull(), [ID, TEXT, TARGET ]]
    df_train['word_count'] = df_train[TEXT].apply(lambda x: len(str(x).split()))
    #print(df_train[df_train[TARGET]==1]['word_count'].mean()) #Positive match
    #print(df_train[df_train[TARGET]==0]['word_count'].mean()) #Not positive match 
    df_train['clean_text'] = df_train[TEXT].apply(lambda x: finalpreprocess(x))
    # create Word2vec model
    #  here words_f should be a list containing words from each document. say 1st row of the list is words from the 1st document/sentence
    #  length of words_f is number of documents/sentences in your dataset
    df_train['clean_text_tok']=[nltk.word_tokenize(i) for i in df_train['clean_text']] #convert preprocessed sentence to tokenized sentence
    model = Word2Vec(df_train['clean_text_tok'],min_count=1)  #min_count=1 means word should be present at least across all documents,
    #if min_count=2 means if the word is present less than 2 times across all the documents then we shouldn't consider it
    w2v = dict(zip(model.wv.index_to_key, model.wv.vectors))  #combination of word and its vector

    #SPLITTING THE TRAINING DATASET INTO TRAINING AND VALIDATION
    # Input: "reviewText", "rating" and "time"
    # Target: "log_votes"
    X_train, X_val, y_train, y_val = train_test_split(df_train["clean_text"],
                                                      df_train[TARGET],
                                                      test_size=0.2,
                                                      shuffle=True)
    X_train_tok= [nltk.word_tokenize(i) for i in X_train]  #for word2vec
    X_val_tok= [nltk.word_tokenize(i) for i in X_val]      #for word2vec
    #TF-IDF
    # Convert x_train to vector since model can only run on numbers and not words- Fit and transform
    tfidf_vectorizer = TfidfVectorizer(use_idf=True)
    X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(X_train) #tfidf runs on non-tokenized sentences unlike word2vec
    # Only transform x_test (not fit and transform)
    X_val_vectors_tfidf = tfidf_vectorizer.transform(X_val) #Don't fit() your TfidfVectorizer to your test data: it will 
    #change the word-indexes & weights to match test data. Rather, fit on the training data, then use the same train-data-
    #fit model on the test data, to reflect the fact you're analyzing the test data only based on what was learned without 
    #it, and the have compatible

    ######################################################################
    
    #FITTING THE CLASSIFICATION MODEL using Logistic Regression(tf-idf)
    lr_tfidf=LogisticRegression(solver = 'liblinear', C=10, penalty = 'l2')
    lr_tfidf.fit(X_train_vectors_tfidf, y_train)  #model

    #Predict y value for test dataset
    y_predict = lr_tfidf.predict(X_val_vectors_tfidf)
    y_prob = lr_tfidf.predict_proba(X_val_vectors_tfidf)[:,1]


    print(classification_report(y_val,y_predict))
    print('Confusion Matrix:',confusion_matrix(y_val, y_predict))

    fpr, tpr, thresholds = roc_curve(y_val, y_prob)
    roc_auc = auc(fpr, tpr)
    print('AUC:', roc_auc)

    ### EXAMINE TEST SET WITH MODEL. This includes the data from training (both pos and neg targets) ###
    df_test['clean_text'] = vd['SYMPTOM_TEXT'].apply(lambda x: finalpreprocess(x)) #preprocess the data
    X_test=df_test['clean_text'] 
    X_vector=tfidf_vectorizer.transform(X_test) #converting X_test to vector
    y_predict = lr_tfidf.predict(X_vector)      #use the trained model on X_vector
    y_prob = lr_tfidf.predict_proba(X_vector)[:,1]
    df_test['predict_prob']= y_prob
    df_test['target']= y_predict
    # display(df_test[['VAERS_ID','SYMPTOM_TEXT','predict_prob','target']].head())
    # #final=df_test[['id','target']].reset_index(drop=True)
    # #final.to_csv('submission.csv')
    df_results = df_test[(
                            (df_test["target"] == 1) & 
                            (df_test["predict_prob"] >= perc_cutoff)
    )]
    num_found = df_results.shape[0]
    num_total = df_test.shape[0]
    print("{}/{} ({})".format(num_found, num_total, num_found/num_total))
    return num_found,df_results



In [13]:
def run_analysis(df, ID, TEXT, TARGET, iterations = 10):
    results = []
    for i in range(0,iterations,1):
        print(); print(" Iteration {} {} ==========".format(i, TEXT))
        df_train = df.loc[vd[TARGET].notnull(), [ID,TEXT,TARGET]]
        df_train['word_count'] = df_train[TEXT].apply(lambda x: len(str(x).split()))
        print("Average word count, positive targets: {}".format(df_train[df_train[TARGET]==1]['word_count'].mean())) #Has covid
        print("Average word count, negative targets: {}".format(df_train[df_train[TARGET]==0]['word_count'].mean())) #Has covid
        num_found,df_results = logistic_regression_tfidf(df_train, df, ID,TEXT,TARGET)
        results.append((df_results, num_found, num_found/df.shape[0]))

    for i in results:
        print(i[1], i[2])
    
    return results


In [14]:
LAB_DATA_RESULTS = run_analysis(vd, 'VAERS_ID','LAB_DATA','LAB_DATA_TARGET')

LAB_SYMPTOM_TEXT_RESULTS = run_analysis(vd, 'VAERS_ID','SYMPTOM_TEXT','SYMPTOM_TEXT_TARGET')


# results = []
# for i in range(0,10,1):
#     df_train = vd.loc[vd[TARGET].notnull(), [ID,TEXT,TARGET]]
#     df_train['word_count'] = df_train[TEXT].apply(lambda x: len(str(x).split()))
#     print("Average word count, positive targets: {}".format(df_train[df_train[TARGET]==1]['word_count'].mean())) #Has covid
#     print("Average word count, negative targets: {}".format(df_train[df_train[TARGET]==0]['word_count'].mean())) #Has covid
#     num_found,df_results_LAB_DATA = logistic_regression_tfidf(df_train, vd, ID,TEXT,TARGET)
#     results.append(num_found/5184)

# LAB_DATA_RESULTS = results
# for i in LAB_DATA_RESULTS:
#     print("{}%".format(i*100))
    
# print("Average: {}%".format( (sum(results)/len(results)) * 100))


 Iteration 0 LAB_DATA ==========
Average word count, positive targets: 19.34285714285714
Average word count, negative targets: 51.73076923076923
              precision    recall  f1-score   support

         0.0       0.91      0.83      0.87        24
         1.0       0.80      0.89      0.84        18

    accuracy                           0.86        42
   macro avg       0.85      0.86      0.86        42
weighted avg       0.86      0.86      0.86        42

Confusion Matrix: [[20  4]
 [ 2 16]]
AUC: 0.9305555555555556
70/5184 (0.013503086419753086)

 Iteration 1 LAB_DATA ==========
Average word count, positive targets: 19.34285714285714
Average word count, negative targets: 51.73076923076923
              precision    recall  f1-score   support

         0.0       0.78      0.82      0.80        22
         1.0       0.79      0.75      0.77        20

    accuracy                           0.79        42
   macro avg       0.79      0.78      0.78        42
weighted avg     

              precision    recall  f1-score   support

         0.0       0.89      0.81      0.85        21
         1.0       0.82      0.90      0.86        20

    accuracy                           0.85        41
   macro avg       0.86      0.85      0.85        41
weighted avg       0.86      0.85      0.85        41

Confusion Matrix: [[17  4]
 [ 2 18]]
AUC: 0.9666666666666667
273/5184 (0.052662037037037035)

 Iteration 5 SYMPTOM_TEXT ==========
Average word count, positive targets: 100.20792079207921
Average word count, negative targets: 104.03960396039604
              precision    recall  f1-score   support

         0.0       0.95      0.95      0.95        19
         1.0       0.95      0.95      0.95        22

    accuracy                           0.95        41
   macro avg       0.95      0.95      0.95        41
weighted avg       0.95      0.95      0.95        41

Confusion Matrix: [[18  1]
 [ 1 21]]
AUC: 0.9688995215311005
252/5184 (0.04861111111111111)

 Iterati

In [26]:
LAB_SYMPTOM_TEXT_RESULTS = LAB_SYMPTOM_TEXT # Named it wrong originally :D Remove for next run :D

results = {"num_covid_infections":[], "ratio_to_vd":[]}
for i in range(0,len(LAB_SYMPTOM_TEXT_RESULTS), 1):     
    print(); print("({})==========================".format(i))
    df1 = LAB_SYMPTOM_TEXT_RESULTS[i][0]
    df1 = df1[df1["target"] == 1]
    print("df_results_SYMPTOM_TEXT=",df1.shape)
    
    df2 = LAB_DATA_RESULTS[i][0]
    df2 = df2[df2["target"] == 1]
    print("df_results_LAB_DATA=",df2.shape)


    print("SYMPTOM_TEXT + LAB_DATA rows = ", df1.shape[0] + df2.shape[0])
    for key in ["clean_text","predict_prob","target"]:
        df1 = df1.drop([key], 1)
        df2 = df2.drop([key], 1)

    df = pd.concat([df1,df2]).drop_duplicates().reset_index(drop=True)

    print("SYMPTOM_TEXT + LAB_DATA, duplicates dropped =", df.shape)
    # Assuming 5% error, so removing 5%
    num_covid_infections = len(df["VAERS_ID"].unique()) * 0.95
    print("Unique VAERS IDs in SYMPTOM_TEXT + LAB_DATA = ",num_covid_infections)
    
    results["num_covid_infections"].append(num_covid_infections)
    results["ratio_to_vd"].append((num_covid_infections/vd.shape[0]))

print()    

print("Average number of covid positives in VAERS data (deaths only): ",  np.mean(results["num_covid_infections"]))
print("Average percent of covid positives in VAERS data (deaths only): ", np.mean(results["ratio_to_vd"])*100)
                   
                   
    



(0)==========================
df_results_SYMPTOM_TEXT= (231, 42)
df_results_LAB_DATA= (70, 42)
SYMPTOM_TEXT + LAB_DATA rows =  301
SYMPTOM_TEXT + LAB_DATA, duplicates dropped = (244, 39)
Unique VAERS IDs in SYMPTOM_TEXT + LAB_DATA =  231.79999999999998

(1)==========================
df_results_SYMPTOM_TEXT= (230, 42)
df_results_LAB_DATA= (61, 42)
SYMPTOM_TEXT + LAB_DATA rows =  291
SYMPTOM_TEXT + LAB_DATA, duplicates dropped = (237, 39)
Unique VAERS IDs in SYMPTOM_TEXT + LAB_DATA =  225.14999999999998

(2)==========================
df_results_SYMPTOM_TEXT= (231, 42)
df_results_LAB_DATA= (50, 42)
SYMPTOM_TEXT + LAB_DATA rows =  281
SYMPTOM_TEXT + LAB_DATA, duplicates dropped = (237, 39)
Unique VAERS IDs in SYMPTOM_TEXT + LAB_DATA =  225.14999999999998

(3)==========================
df_results_SYMPTOM_TEXT= (269, 42)
df_results_LAB_DATA= (70, 42)
SYMPTOM_TEXT + LAB_DATA rows =  339
SYMPTOM_TEXT + LAB_DATA, duplicates dropped = (277, 39)
Unique VAERS IDs in SYMPTOM_TEXT + LAB_DATA =  263

# Methods:
The VAERS data was downloaded for all of 2021, first week of July from https://vaers.hhs.gov/data.html. Data was filtered to include only rows where column "DIED" == "Y".

Two columns were pertinent to the disucsion: "SYMPTOM_TEXT" and "LAB_DATA". Information verifying that patient had covid may appear in one or both columns, so each column was processed separately. The results of each search were then concatenated together, and duspliactes were removed based on "VAERS_ID". The row counts were then tallied.  

The model was trained by manually adding the columns: "SYMPTOM_TEXT_TARGET" and "LAB_DATA_TARGET". These columns were manually marked with a 0 (not infected with covid) or 1 (infected with covid) based on the contents of the "SYMPTOM_TEXT" and "LAB_DATA" columns (respectively.) A pateint was considered "Infected with covid" if they had a positive test within 30 days prior to the vaccination, after shot, or if the text annotated that the patient ad covid symptoms or died of a covid related symptom.

The model was trained the dataset using Logistic Regression(tf-idf), and then ran the model against the entire death-only VAERS data, including the data from the training set (in order to capture the total of all patients who had, developed, or died of covid.)

101 positive (1s) and 101 negative (0s) were marked for SYMPTOM_TEXT_TARGET
105 positive (1s) and 104 negative (0s) were marked for LAB_DATA_TARGET

From a randomly selected run from "SYMPTOM TEXT", 12 out of 233 were incorrect (0.0482)
Assuming a 5% error rate when using .80 from the "predict_prob" column as the cutoff 


# Results:
Out of 5184 reports of deaths after vaccination ...<br>

**Average number of covid positives in VAERS data (deaths only):  240.06499999999997**

**Average percent of covid positives in VAERS data (deaths only):  4.63088348765432**


